In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from compare_paths import *
from utils import *
from pytorch_value_iteration_networks.model import *
from types import SimpleNamespace
from eval import *
import pickle
import pandas as pd 

# VIN path compare

In [2]:

df = pd.read_csv("path_results.csv")

In [3]:
df.head()

,seed,algorithm,mean_inference_time,total_steps,successful,path,path_vin
0,16494,VI,0.002660,7,True,"[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4...",NaN
1,16494,VIN,0.000466,7,True,NaN,"[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4..."
2,6545,VI,0.003278,12,True,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",NaN
3,6545,VIN,0.000392,12,True,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5..."
4,19842,VI,0.002788,17,True,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0...",NaN


In [17]:
fails = df[df["successful"]==True]
#fails["seed"].sample(10).to_list()


[14865, 14009, 16276, 12884, 17928, 9883, 11524, 17155, 13482, 17610]

In [18]:
successes = df[df["successful"]==True]

In [ ]:
successes.sample(10)["seed"].to_list()

In [40]:
vins = df[df["algorithm"]=="VIN"]

print(vins["total_steps"].mean())


17.691756272401435


In [41]:
vis = df[df["algorithm"]=="VI"]
print(vis["successful"].mean())
print(vis["mean_inference_time"].mean())
print(vis["total_steps"].mean())

0.985663082437276
0.002423629721919019
17.65232974910394
